## Dependencies
View installation instructions for the OMERO library on the [OME website](https://www.openmicroscopy.org/site/support/omero5.2/developers/Python.html).

In [23]:
%matplotlib inline

from collections import defaultdict
import math
import os
import re
import struct

import matplotlib.pyplot as plt
import numpy as np
from scipy.ndimage import imread
from skimage.measure import regionprops

import omero
from omero.gateway import BlitzGateway
from omero.rtypes import rdouble, rint

## Connect to the OMERO server
First, we connect to the OMERO server instance. Replace the username and password credentials with your own, and if necessary, set the correct host and port of the OMERO server.

In [24]:
username = 'root'
password = 'password'

conn = BlitzGateway(username, password, host='localhost', port=4064, secure=True)
conn.connect()

roi_service = conn.getRoiService()
update_service = conn.getUpdateService()

## Choose an image to add ROIs to

Below, replace the image ID (can be found in the OMERO web client -- select an image, and look for the ID in the panel on the right of the page, near the top). Also, specify the covertrack object and outline directories for the image.

In [25]:
image_id = 151
object_dir = '/home/snguyen/covert/output/20160701/Pos1/objects'
outline_dir = '/home/snguyen/covert/output/20160701/Pos1/outlines'

In [26]:
def get_plane(filename):
    pattern = re.compile('^img_channel(?P<c>\d{3})_position\d{3}_time(?P<t>\d{9})_z(?P<z>\d{3})_(?P<obj_type>.+)\.png$')
    match = pattern.match(object_file)
    c = int(match.group('c'))
    t = int(match.group('t'))
    z = int(match.group('z'))
    obj_type = match.group('obj_type')
    
    return z, t, c, obj_type


def create_mask_bytes(mask_img, bytes_per_pixel=1):
    mask_bytes = mask_img.astype(np.uint8).tostring()
    
    # From https://github.com/openmicroscopy/openmicroscopy/blob/v5.2.7/examples/Training/python/ROIs.py#L115
    if bytes_per_pixel == 2:
        divider = 16.0
        format_string = "H"  # Unsigned short
        byte_factor = 0.5
    elif bytes_per_pixel == 1:
        divider = 8.0
        format_string = "B"  # Unsigned char
        byte_factor = 1
    else:
        message = "Format %s not supported"
        raise ValueError(message)
    steps = math.ceil(len(mask_bytes) / divider)
    mask = []
    for i in range(long(steps)):
        binary = mask_bytes[
            i * int(divider):i * int(divider) + int(divider)]
        format = str(int(byte_factor * len(binary))) + format_string
        binary = struct.unpack(format, binary)
        s = ""
        for bit in binary:
            s += str(bit)
        mask.append(int(s, 2))
    return bytearray(mask)


def create_omero_mask(mask_img, bounding_box, z, t, c):
    min_x, min_y, max_x, max_y = bounding_box
    
    mask = omero.model.MaskI()
    mask.setTheC(rint(c))
    mask.setTheZ(rint(z))
    mask.setTheT(rint(t))
    mask.setBytes(create_mask_bytes(mask_img[min_x:max_x, min_y:max_y]))

    # This flipping of x and y is required, otherwise the mask ends up mirrored
    # across the diagonal of the image.
    mask.setX(rdouble(min_y))
    mask.setY(rdouble(min_x))
    mask.setWidth(rdouble(max_y - min_y))
    mask.setHeight(rdouble(max_x - min_x))
    
    return mask


# From https://github.com/openmicroscopy/openmicroscopy/blob/v5.2.7/examples/Training/python/ROIs.py#L54
def save_roi(img, shapes):
    # create an ROI, link it to Image
    roi = omero.model.RoiI()
    # use the omero.model.ImageI that underlies the 'image' wrapper
    roi.setImage(img._obj)
    for shape in shapes:
        roi.addShape(shape)
    # Save the ROI (saves any linked shapes too)
    update_service.saveAndReturnObject(roi)

## Add ROIs
Simply run the cell below. If you want to add outlines instead of objects, replace `object_dir` with `outline_dir` as the iteration source in the outer `for` loop. This may take some time to run, depending on the number of ROIs and the number of images to process.

In [28]:
# TODO Consider using multiprocessing here. Caution -- defaultdict may not be thread-safe, so 
# might have to rework the mask storage mechanism a bit.

masks = defaultdict(list)
max_object_id = None

for object_file in sorted(os.listdir(object_dir)):    
    z, t, c, obj_type = get_plane(object_file)
    objects_mask = imread(os.path.join(object_dir, object_file))
    
    # FIXME Should there be a max for each object_type / c?
    plane_max_object_id = np.max(objects_mask)
    max_object_id = plane_max_object_id if max_object_id is None else max(max_object_id, plane_max_object_id)
    
    for object_id in xrange(1, max_object_id):
        object_mask = objects_mask == object_id
        if np.max(object_mask) == 1:
            regions = regionprops(object_mask.astype(np.int))
            if len(regions) != 1:
                print 'Warning: multiple regions for object id', object_id
            else:
                mask = create_omero_mask(object_mask, regions[0].bbox, z, t, c)
                masks[object_id].append(mask)

                
image = conn.getObject('Image', image_id)
for object_id in masks:
    save_roi(image, masks[object_id])

## Delete all ROIs on an image


In [27]:
result = roi_service.findByImage(image_id, None)
conn.deleteObjects('Roi', [roi.id.val for roi in result.rois], wait=True)

1a794520-abc3-437a-9ef9-e155a20d3406/IHandle11e8faec-cd1e-4df7-8ec7-ffaa11c84db8 -t -e 1.0:tcp -h 10.210.44.34 -p 58651 -t 60000:tcp -h 10.210.30.91 -p 58651 -t 60000

## More information
Further example code can be found on OME's Github repo. Example code for handling ROIs can be found [here](https://github.com/openmicroscopy/openmicroscopy/blob/v5.2.7/examples/Training/python/ROIs.py).